# Building an Agent with Generative Models and Tools

In [ ]:
%pip install openai pinecone


[notice] A new release of pip available: 22.2.2 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [6]:
import datetime
import re
from copy import copy
from functools import lru_cache

import sys
from io import StringIO
from typing import Dict, Optional, Any
from openai import OpenAI
import os

from pydantic import BaseModel, Field
from typing import List, Dict, Tuple

In [7]:
import datetime
import re
from copy import copy

import sys
from io import StringIO
from typing import Dict, Optional, Any
import os

from pydantic import BaseModel, Field
from typing import List, Dict, Tuple
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

True

In [8]:
# url: str = userdata.get('SUPABASE_URL')
# key: str = userdata.get('SUPABASE_API_KEY')
# supabase: Client = create_client(url, key)

# Initialize the OpenAI client with the API key from user data
client = OpenAI(
    api_key=os.getenv('OPENAI_API_KEY')
)

class ChatLLM(BaseModel):
    # Wrapper on top of OpenAI API
    model: str = 'gpt-4o'
    temperature: float = 0.0

    def generate(self, prompt: str, stop: List[str] = None):
        response = client.chat.completions.create(
            model=self.model,
            messages=[{"role": "user", "content": prompt}],
            temperature=self.temperature,
            stop=stop
        )
        # Insert cost projection into postgre db 

        # supabase.table('cost_projecting').insert({
        #     'prompt': prompt,
        #     'response': response.choices[0].message.content,
        #     'input_tokens': response.usage.prompt_tokens,
        #     'output_tokens': response.usage.completion_tokens,
        #     'model': self.model,
        #     'inference_params' : {
        #         'temperature': self.temperature,
        #         'stop': stop
        #     },
        #     'is_openai': True,
        #     'app': 'RAG'
        # }).execute()

        return response.choices[0].message.content

In [9]:
llm = ChatLLM()
result = llm.generate(prompt='Who is the president of Turkey?')
print(result)

As of my last update, the president of Turkey is Recep Tayyip Erdoğan. He has been in office since August 28, 2014. However, please verify with a current source, as political positions can change.


In [ ]:
from pinecone import Pinecone, ServerlessSpec
# Retrieve the Pinecone API key from user data
pinecone_key = os.getenv('PINECONE_API_KEY')

# Define constants for the Pinecone index, namespace, and engine
INDEX_NAME = 'semantic-search-rag'  # The name of the Pinecone index
NAMESPACE = 'default'  # The namespace to use within the index
ENGINE = 'text-embedding-3-small'  # The embedding model to use (vector size 1,536)

# Initialize the Pinecone client with the retrieved API key
pc = Pinecone(
    api_key=pinecone_key
)

In [11]:
# helper functions to get embedding from the OpenAI API
def get_embedding(texts, engine=ENGINE):
    response = client.embeddings.create(
        input=texts,
        model=engine
    )

    return response.data[0].embedding



len(get_embedding('hi'))

1536

In [13]:
if INDEX_NAME not in pc.list_indexes().names():  # need to create the index
    print(f'Creating index {INDEX_NAME}')
    pc.create_index(
        name=INDEX_NAME,  # The name of the index
        dimension=1536,  # The dimensionality of the vectors for our OpenAI embedder
        metric='cosine',  # The similarity metric to use when searching the index
        spec=ServerlessSpec(
            cloud='aws',
            region='us-east-1'
        )
    )

# Store the index as a variable
index = pc.Index(name=INDEX_NAME)
index

/Users/pnoras/Repositories/python/nlp/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [15]:
def query_from_pinecone(query, top_k=3, include_metadata=True):
    # get embedding from THE SAME embedder as the documents
    query_embedding = get_embedding(query, engine=ENGINE)

    return index.query(
      vector=query_embedding,
      top_k=top_k,
      namespace=NAMESPACE,
      include_metadata=include_metadata   # gets the metadata (dates, text, etc)
    ).get('matches')

query_from_pinecone('I lost my medicare card')[0]['metadata']

{'date_uploaded': '2025-08-31T10:07:55.396448+00:00',
 'text': '\n\n\n\nYou’re offline. This is a read only version of the page.\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSkip to content\n\n\n  \n\n\n\n\n\n\n\nProtect Yourself from Scams \n\n\n\n \n\n \n\n\n\n\nProtect Yourself from Scams\n\n\n\nSkip to main content Social Security Search  Menu  Español  Sign in\n\n\n\n\nFrequently Asked Questions\n\n\n\n\nLast Modified: \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nFAQ Home\n\n\nTopics\n\n\r\n\t\t\t\t\tKA-01735\r\n\t\t\t\t\n\n\n\n\n\n Print\n\n\n\nHow do I get a replacement Medicare card? \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nViews: \n\n\n\nIf your Medicare card was lost, stolen, or destroyed, you can request a replacement online at Medicare.gov.\nYou can print an official copy of your card from your online Medicare account \nor call 1-800-MEDICARE (1-800-633-4227 TTY 1-877-486-2048) to order a replacement card to be sent in the mail.\n\n\n\n\n\n\n\n\n\n\n\n\n

In [29]:
FINAL_ANSWER_TOKEN = "Assistant Response:"
STOP = '[END]'
PROMPT_TEMPLATE = """Today is {today} and you can retrieve information from a database. Response the user's input as best as you can.

Here is an example of the conversation format:

[START]
User Input: the input question you must answer
Context: retrieved context from the database
Context URL: context url
Context Score : a score from 0 - 1 of how strong the information is a match
Assistant Thought: This context has sufficient information to answer the question.
Assistant Response: your final answer to the original input question which could be I don't have sufficient information to answer the question and you should add the context url if having sufficient information.
[END]
[START]
User Input: another input question you must answer
Context: more retrieved context from the database
Context URL: context url
Context Score : another score from 0 - 1 of how strong the information is a match
Assistant Thought: This context does not have sufficient information to answer the question.
Assistant Response: your final answer to the second input question which could be I don't have sufficient information to answer the question. and you should add the context url if having sufficient information. 
[END]
[START]
User Input: another input question you must answer
Context: NO CONTEXT FOUND
Context URL: NONE
Context Score : 0
Assistant Thought: We either could not find something or we don't need to look something up
Assistant Response: Reason through the best way to answer
[END]

Begin:

{running_convo} 
"""

In [30]:
class RagBot(BaseModel):
    llm: ChatLLM
    prompt_template: str = PROMPT_TEMPLATE
    stop_pattern: List[str] = [STOP]
    user_inputs: List[str] = []
    ai_responses: List[str] = []
    contexts: List[Tuple[str,float]] = []
    verbose: bool = False
    threshold: float = 0.6

    def query_from_pinecone(self, query: str, top_k: int = 3, include_metadata: bool = True) -> List[Tuple[str,float]]:
        """Query Pinecone for the most relevant contexts"""
        return query_from_pinecone(query, top_k, include_metadata)
    
    @property
    def running_convo(self):
        convo = ''
        for index in range(len(self.user_inputs)):
            convo += f'[START]\nUser Input: {self.user_inputs[index]}\n'
            convo += f'Context: {self.contexts[index][0]}\nContext URL: {self.contexts[index][1]}\nContext Score: {self.contexts[index][2]}'

            if len(self.ai_responses) > index:
                convo += self.ai_responses[index]
                convo += '\n[END]\n'
        return convo
    
    def run(self, question: str) -> str:
        """Run the RAG bot"""
        
        self.user_inputs.append(question)
        top_response = self.query_from_pinecone(question)[0]

        print(top_response['score'])

        if top_response['score'] > self.threshold:
            self.contexts.append(
                (top_response['metadata']['text'], top_response['metadata']['url'], top_response['score'])
            )  
        
        else:
            self.contexts.append(
                ('NO CONTEXT FOUND', 'NONE', 0)
            )
        
        prompt = self.prompt_template.format(
            today=datetime.datetime.now().strftime("%Y-%m-%d"),
            running_convo=self.running_convo
        )

        if self.verbose:
            print('-------')
            print('PROMPT')
            print('-------')
            print(prompt)
            print('-------')

        generated = self.llm.generate(prompt, self.stop_pattern)

        if self.verbose:
            print('-------')
            print('GENERATED')
            print('-------')
            print(generated)
            print('-------')

        self.ai_responses.append(generated)
        if FINAL_ANSWER_TOKEN in generated:
            generated = generated.split(FINAL_ANSWER_TOKEN)[-1]

        return generated         

In [31]:
r = RagBot(llm=ChatLLM(temperature=0.0), stop_pattern=['[END]'])
print(r.run('I lost my medicare card'))

0.693827927
 If your Medicare card was lost, stolen, or destroyed, you can request a replacement online at Medicare.gov. You can also print an official copy of your card from your online Medicare account or call 1-800-MEDICARE (1-800-633-4227 TTY 1-877-486-2048) to order a replacement card to be sent in the mail. For more information, you can visit [this link](https://faq.ssa.gov/en-us/Topic/article/KA-01735).


In [32]:
print(r.running_convo)

[START]
User Input: I lost my medicare card
Context: 



You’re offline. This is a read only version of the page.































Skip to content


  







Protect Yourself from Scams 



 

 




Protect Yourself from Scams



Skip to main content Social Security Search  Menu  Español  Sign in




Frequently Asked Questions




Last Modified: 

















FAQ Home


Topics


					KA-01735
				





 Print



How do I get a replacement Medicare card? 

























Views: 



If your Medicare card was lost, stolen, or destroyed, you can request a replacement online at Medicare.gov.
You can print an official copy of your card from your online Medicare account 
or call 1-800-MEDICARE (1-800-633-4227 TTY 1-877-486-2048) to order a replacement card to be sent in the mail.


































Footer menu








Give us Feedback.

Did this answer your question?

No
Yes
No

Thanks for your feedback.







Context URL: https://faq.ssa.gov/en-us/